Hello! I'm Keshav and this is my attempt at analyzing the varying sentiments of news data.

For some context, I found this dataset on Kaggle titled "Facebook Posts from ABC, BBC, CBS, CNN (2012-2016)" by Rishi Damarla. For the purpose of this model, I narrowed the focus to CNN. This also indicates the potential for a future project that compares the sentiments across news organizations (i.e. how CNN or Fox differs in the words that result in the greatest positive ratings).

The first step is to import relevant libraries and set up the connection with google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip3 install spacy==3.2.0
!pip3 install spacytextblob
!python3 -m spacy download en_core_web_sm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
2024-03-19 06:12:16.923144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 06:12:16.923244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 06:12:16.925852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 06:12:16.940176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA,

In [ ]:
df_cnn =pd.read_csv('gdrive/MyDrive/News/cnn.csv')

In [ ]:
df_cnn.head()

,id,page_id,name,message,description,caption,post_type,status_type,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count,link,picture,posted_at
0,"﻿""5550296508_10150712177946509""",5550296508,NaN,Breaking News: French prosecutors: Former IMF ...,NaN,NaN,status,mobile_status_update,348,143,201,0,0,0,0,0,0,NaN,NaN,2012-03-26 21:27:01
1,"﻿""5550296508_258636547563092""",5550296508,How nations risk nuclear terrorism - CNN.com,CNN Opinion Contributor Richard Chasdi states ...,Richard Chasdi says that nations that empower ...,cnn.com,link,shared_story,542,118,71,0,0,0,0,0,0,http://www.cnn.com/2012/03/26/opinion/chasdi-n...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-03-26 21:30:27
2,"﻿""5550296508_10150712540566509""",5550296508,Facebook wants court to dismiss Ceglia lawsuit,"Ceglia has forged documents, destroyed evidenc...",Paul Ceglia originally filed the attention-gra...,money.cnn.com,link,published_story,185,67,0,0,0,0,0,0,0,http://cnnmon.ie/HdBpwG,https://external.xx.fbcdn.net/safe_image.php?d...,2012-03-27 1:30:45
3,"﻿""5550296508_350156181698587""",5550296508,Report: Zimmerman told police teen punched him...,An Orlando Sentinel report fills in some blank...,"A month ago Monday, Trayvon Martin, an unarmed...",cnn.com,link,shared_story,488,3009,124,0,0,0,0,0,0,http://www.cnn.com/2012/03/26/justice/florida-...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-03-27 13:41:26
4,"﻿""5550296508_140431756086124""",5550296508,Supreme Court divided over health care mandate...,At the core of the health care law is the indi...,The Supreme Court appeared divided Tuesday ove...,cnn.com,link,shared_story,538,782,125,0,0,0,0,0,0,http://www.cnn.com/2012/03/27/justice/scotus-h...,https://external.xx.fbcdn.net/safe_image.php?d...,2012-03-27 17:52:15


In [ ]:
df_cnn.describe()

,page_id,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count
count,3.169600e+04,3.169600e+04,31696.000000,3.169600e+04,31696.000000,31696.000000,31696.000000,31696.000000,31696.000000,31696.000000
mean,5.550297e+09,4.218814e+03,742.776218,1.460405e+03,134.203149,71.098088,85.498265,114.827139,0.067674,84.207566
std,0.000000e+00,1.318657e+04,1964.627591,1.440600e+04,1239.543133,564.055115,576.071482,1554.700428,3.691511,870.131719
min,5.550297e+09,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.550297e+09,9.910000e+02,148.000000,1.380000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.550297e+09,1.960000e+03,334.000000,3.530000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.550297e+09,4.028250e+03,809.000000,9.600000e+02,19.000000,21.000000,8.000000,5.000000,0.000000,5.000000
max,5.550297e+09,1.155249e+06,237266.000000,1.934157e+06,156066.000000,45708.000000,34904.000000,149098.000000,509.000000,117430.000000


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


[('This', 'PRON'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]


In [ ]:
doc = nlp("How nations risk nuclear terrorism - CNN.com")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


How how SCONJ WRB advmod Xxx True True
nations nation NOUN NNS nsubj xxxx True False
risk risk VERB VBP ROOT xxxx True False
nuclear nuclear ADJ JJ amod xxxx True False
terrorism terrorism NOUN NN compound xxxx True False
- - PUNCT HYPH punct - False False
CNN.com CNN.com PROPN NNP dobj XXX.xxx False False


1] Data Cleaning


In [ ]:
df_cnn.head().T

,0,1,2,3,4
id,"﻿""5550296508_10150712177946509""","﻿""5550296508_258636547563092""","﻿""5550296508_10150712540566509""","﻿""5550296508_350156181698587""","﻿""5550296508_140431756086124"""
page_id,5550296508,5550296508,5550296508,5550296508,5550296508
name,NaN,How nations risk nuclear terrorism - CNN.com,Facebook wants court to dismiss Ceglia lawsuit,Report: Zimmerman told police teen punched him...,Supreme Court divided over health care mandate...
message,Breaking News: French prosecutors: Former IMF ...,CNN Opinion Contributor Richard Chasdi states ...,"Ceglia has forged documents, destroyed evidenc...",An Orlando Sentinel report fills in some blank...,At the core of the health care law is the indi...
description,NaN,Richard Chasdi says that nations that empower ...,Paul Ceglia originally filed the attention-gra...,"A month ago Monday, Trayvon Martin, an unarmed...",The Supreme Court appeared divided Tuesday ove...
caption,NaN,cnn.com,money.cnn.com,cnn.com,cnn.com
post_type,status,link,link,link,link
status_type,mobile_status_update,shared_story,published_story,shared_story,shared_story
likes_count,348,542,185,488,538
comments_count,143,118,67,3009,782


In [ ]:
df_cnn = df_cnn.drop(['id', 'page_id', 'caption', 'post_type', 'link', 'picture','posted_at', 'status_type'], axis=1)

In [ ]:
df_cnn.head()

,name,message,description,likes_count,comments_count,shares_count,love_count,wow_count,haha_count,sad_count,thankful_count,angry_count
0,NaN,Breaking News: French prosecutors: Former IMF ...,NaN,348,143,201,0,0,0,0,0,0
1,How nations risk nuclear terrorism - CNN.com,CNN Opinion Contributor Richard Chasdi states ...,Richard Chasdi says that nations that empower ...,542,118,71,0,0,0,0,0,0
2,Facebook wants court to dismiss Ceglia lawsuit,"Ceglia has forged documents, destroyed evidenc...",Paul Ceglia originally filed the attention-gra...,185,67,0,0,0,0,0,0,0
3,Report: Zimmerman told police teen punched him...,An Orlando Sentinel report fills in some blank...,"A month ago Monday, Trayvon Martin, an unarmed...",488,3009,124,0,0,0,0,0,0
4,Supreme Court divided over health care mandate...,At the core of the health care law is the indi...,The Supreme Court appeared divided Tuesday ove...,538,782,125,0,0,0,0,0,0


In [ ]:
df_cnn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31696 entries, 0 to 31695
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            29816 non-null  object
 1   message         31091 non-null  object
 2   description     18797 non-null  object
 3   likes_count     31696 non-null  int64 
 4   comments_count  31696 non-null  int64 
 5   shares_count    31696 non-null  int64 
 6   love_count      31696 non-null  int64 
 7   wow_count       31696 non-null  int64 
 8   haha_count      31696 non-null  int64 
 9   sad_count       31696 non-null  int64 
 10  thankful_count  31696 non-null  int64 
 11  angry_count     31696 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 2.9+ MB


In [ ]:
count = 0;
for i in df_cnn['description']:
  if count > 10:
    break
  print(i);
  print("\n\n");
  count += 1

nan



Richard Chasdi says that nations that empower proxy groups are risking disaster in a nuclear age



Paul Ceglia originally filed the attention-grabbing lawsuit in July 2010, claiming he's owed a 50% ownership stake in Facebook.



A month ago Monday, Trayvon Martin, an unarmed Florida teen, died at the hands of a neighborhood watch volunteer, stirring outrage and protests nationwide.



The Supreme Court appeared divided Tuesday over the most controversial provision of the 2010 federal health care law.



nan



nan



These 5 creations look to be some of this year's most transformative tech products and services.



When health care reform passed Congress more than two years ago, Julie Walters yelled for her husband to come into the living room where she was watching the vote live on television.



nan



Washington (CNN) -- Newt Gingrich has replaced his campaign manager and is laying off one-third of his staff, but has vowed to continue his bid for the Republican presidential

Description is missing in a substantive amount, almost half, of all the data sources. Based on the text of description, it seems relatively important when explaining information in the context of the sources. This leaves us with 3 options:
(1) Eliminate the rows with description
(2) Work with half the rows as Nan
(3) Potentially replace the description values that are null with the name of the article which does serve as an indicator of the article's content


In [ ]:
for index, row in df_cnn.iterrows():
  if pd.isnull(df_cnn.loc[index, 'description']):
    df_cnn.at[index,'description'] = df_cnn.loc[index, 'name']


In [ ]:
df_cnn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31696 entries, 0 to 31695
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            29816 non-null  object
 1   message         31091 non-null  object
 2   description     29816 non-null  object
 3   likes_count     31696 non-null  int64 
 4   comments_count  31696 non-null  int64 
 5   shares_count    31696 non-null  int64 
 6   love_count      31696 non-null  int64 
 7   wow_count       31696 non-null  int64 
 8   haha_count      31696 non-null  int64 
 9   sad_count       31696 non-null  int64 
 10  thankful_count  31696 non-null  int64 
 11  angry_count     31696 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 2.9+ MB


In [ ]:
df_cnn = df_cnn.dropna()
df_cnn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29211 entries, 1 to 31695
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            29211 non-null  object
 1   message         29211 non-null  object
 2   description     29211 non-null  object
 3   likes_count     29211 non-null  int64 
 4   comments_count  29211 non-null  int64 
 5   shares_count    29211 non-null  int64 
 6   love_count      29211 non-null  int64 
 7   wow_count       29211 non-null  int64 
 8   haha_count      29211 non-null  int64 
 9   sad_count       29211 non-null  int64 
 10  thankful_count  29211 non-null  int64 
 11  angry_count     29211 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 2.9+ MB


In [ ]:
df_temp = df_cnn['description']
df_temp.head()

1    Richard Chasdi says that nations that empower ...
2    Paul Ceglia originally filed the attention-gra...
3    A month ago Monday, Trayvon Martin, an unarmed...
4    The Supreme Court appeared divided Tuesday ove...
7    These 5 creations look to be some of this year...
Name: description, dtype: object

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [ ]:
text = df_cnn['description'].tolist()
text_score = []
text_label = []
total_pos = []
total_neg = []

In [ ]:
import random
page_text = ""
for index, row in df_cnn.iterrows():
  num = random.randint(1, 100)
  if num == 2:
    page_text += df_cnn.loc[index, 'description']

doc = nlp(page_text)
sentiment = doc._.blob.polarity
sentiment = round(sentiment,2)

if sentiment > 0:
  sent_label = "Positive"
else:
  sent_label = "Negative"

text_label.append(sent_label)
text_score.append(sentiment)

In [ ]:
positive_words = []
negative_words = []

for x in doc._.blob.sentiment_assessments.assessments:
  if x[1] > 0:
    positive_words.append(x[0][0])
  elif x[1] < 0:
    negative_words.append(x[0][0])
  else:
    pass

total_pos.append(', '.join(set(positive_words)))
total_neg.append(', '.join(set(negative_words)))

In [ ]:
df_temp["Sentiment Score"] = text_score
df_temp["Sentiment Label"] = text_label
df_temp["Positive Words"] = total_pos
df_temp["Negative Words"] = total_neg

#optional export to CSV'
df_temp.to_csv("sentiment.csv")
df_temp

1                  Richard Chasdi says that nations that empower ...
2                  Paul Ceglia originally filed the attention-gra...
3                  A month ago Monday, Trayvon Martin, an unarmed...
4                  The Supreme Court appeared divided Tuesday ove...
7                  These 5 creations look to be some of this year...
                                         ...                        
31695                                                               
Sentiment Score                                               [0.08]
Sentiment Label                                           [Positive]
Positive Words     [ceremonial, sexually, far, very, major, best,...
Negative Words     [black, killed, significantly, few, dead, alle...
Name: description, Length: 29215, dtype: object